In [1]:
# import requests
# import folium
# import pandas as pd


# url = 'http://api.vworld.kr/req/address?'
# params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
# road_type = 'ROAD'  # 도로명 주소
# road_type2 = 'PARCEL'  # 지번 주소
# address = '&address='
# keys = '&key='
# primary_key = 'B6B30F8F-B61E-3BBE-889D-A0A9638F7CB9'

# # 도로명
# def request_geo(road):
#     page = requests.get(url + params + road_type + address + road + keys + primary_key)
#     json_data = page.json()
#     if json_data['response']['status'] == 'OK':
#         x = json_data['response']['result']['point']['x']
#         y = json_data['response']['result']['point']['y']
#         return y, x
#     else:
#         return 0, 0

# # 지번
# def request_geo2(parcel):
#     page = requests.get(url + params + road_type2 + address + parcel + keys + primary_key)
#     json_data = page.json()
#     if json_data['response']['status'] == 'OK':
#         x = json_data['response']['result']['point']['x']
#         y = json_data['response']['result']['point']['y']
#         return y, x
#     else:
#         return 0, 0


In [3]:
import requests
import folium
import pandas as pd

url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'  # 도로명 주소
road_type2 = 'PARCEL'  # 지번 주소
address = '&address='
keys = '&key='
primary_key = 'B6B30F8F-B61E-3BBE-889D-A0A9638F7CB9'

# 도로명
def request_geo(road):
    try:
        page = requests.get(url + params + road_type + address + road + keys + primary_key)
        page.raise_for_status()  # Raises an error for bad responses
        json_data = page.json()
        if json_data['response']['status'] == 'OK':
            x = json_data['response']['result']['point']['x']
            y = json_data['response']['result']['point']['y']
            return y, x
        else:
            print(f"Error: {json_data['response']['status']}")
            return 0, 0
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return 0, 0
    except ValueError as e:
        print(f"Invalid JSON response: {e}")
        return 0, 0

# 지번
def request_geo2(parcel):
    try:
        page = requests.get(url + params + road_type2 + address + parcel + keys + primary_key)
        page.raise_for_status()  # Raises an error for bad responses
        json_data = page.json()
        if json_data['response']['status'] == 'OK':
            x = json_data['response']['result']['point']['x']
            y = json_data['response']['result']['point']['y']
            return y, x
        else:
            print(f"Error: {json_data['response']['status']}")
            return 0, 0
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return 0, 0
    except ValueError as e:
        print(f"Invalid JSON response: {e}")
        return 0, 0

# File handling
try:
    df = pd.read_csv('Data1.csv', encoding='cp949')
except FileNotFoundError:
    print("The file 'Data1.csv' was not found.")
    raise

# Handle user input
price = 0
search_value = input("Please enter the address: ")

# Create a new column with concatenated values from 3rd and 6th columns
df['concatenated'] = df.iloc[:, 2].astype(str) + " " + df.iloc[:, 5].astype(str)

# Matching address
result = df[df['concatenated'] == search_value]

if result.empty:
    print(f"{search_value} not found")
    x, y = 0, 0  # Default coordinates if not found
else:
    parcel_address = result['concatenated'].values[0]
    x, y = request_geo2(parcel_address)
    price = result.iloc[0, 8]  # Adjust index if necessary

# Map visualization
def viz_map(x, y):
    m = folium.Map(
        location=[x, y],
        zoom_start=17,
        width='100%',
        height='100%',
    )
    # Marker
    folium.Marker([x, y], popup=str(price), tooltip=str(price)).add_to(m)
    return m

# Display the map
m = viz_map(x, y)
m.save("map.html")


Please enter the address:  서울특별시 종로구 청운동 1


Request failed: Expecting value: line 1 column 1 (char 0)
